# Upscaling Service - Proof of Concept
This notebooks showcases a demo of the APEx Upscaling Service by demonstrating the capabilities of the [APEx Dispatcher API](https://github.com/ESA-APEx/apex_dispatch_api). In this notebook we will perform a small upscaling exercise for one of the services in the [APEx Algoritm Services Catalogue](https://algorithm-catalogue.apex.esa.int/), specfically the [Wind Turbine Detection](https://algorithm-catalogue.apex.esa.int/apps/wind_turbine_detection#execution-information). We will split up an area of interest in a 20x20km grid and execute this  through this upscaling task through the APEx Dispatch API.

In [1]:
import requests
import rasterio
import numpy as np
import tempfile
import asyncio
import json
import websockets
import httpx
import io
import base64
from ipyleaflet import ImageOverlay
from PIL import Image
from ipyleaflet import Map, GeoJSON, TileLayer
from rasterio.warp import transform_bounds
from shapely.geometry import shape

## Definition of parameters

In [2]:
dispatch_api = "localhost:8000"

In [3]:
application = "https://raw.githubusercontent.com/ESA-APEx/apex_algorithms/main/algorithm_catalog/dhi/wind_turbine/openeo_udp/wind_turbine.json"
endpoint = "https://openeo.vito.be"

In [5]:
spatial_extent =   {
        "coordinates": [
          [
            [
              14.370712654502597,
              47.27563273620049
            ],
            [
              14.370712654502597,
              47.26583764118868
            ],
            [
              14.405670947432327,
              47.26583764118868
            ],
            [
              14.405670947432327,
              47.27563273620049
            ],
            [
              14.370712654502597,
              47.27563273620049
            ]
          ]
        ],
        "type": "Polygon"
      }
year = 2024

In [19]:
# Map related settings
center = shape(spatial_extent).centroid
zoom = 14

## Retrieval of the tiles
The first step in our upscaling exercise is to determine the different tiles to be processed based on the given `area_of_interest`. In this example we ask the dispatcher to split up the area in a `20x20km` grid. This results in a list of tiles that are visualised on the map.

In [6]:
tiles = requests.post(f"http://{dispatch_api}/tiles", json={
    "grid": "20x20km",
    "aoi": spatial_extent
}).json()
print(f"Processing {len(tiles['geometries'])} tiles for area of interest")

Processing 1 tiles for area of interest


In [20]:
# Create a map centered at the approximate center of the area of interest
m = Map(center=[center.y, center.x], zoom=zoom)
 
# Add the tiles (GeometryCollection) to the map
geo_json = GeoJSON(data=tiles)
m.add_layer(geo_json)

# Display the map
m

Map(center=[47.27073518869459, 14.388191800967462], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Launching the upscaling task

Next we trigger the upscaling task on the dispatcher. We provide the details of the processing jobs that need to be executed together with a `dimension`. This is an important parameter as this lets the dispatcher know how to scale up. In this case we are asking the dispatcher to scale up using the `spatial_extent`, creating a separate job for each geometry in the `values` section. The dispatcher will take care of all the rest. The result in the information on the created upscaling task.

In [38]:
upscaling_task = requests.post(f"http://{dispatch_api}/upscale_tasks", json={
    "title": "Wind Turbine Detection",
    "label": "openeo",
    "service": {
        "endpoint": endpoint,
        "application": application
    },
    "parameters": {
        "year": year
    },
    "dimension": {
        "name": "spatial_extent",
        "values": tiles["geometries"]
    }
}).json()
upscaling_task_id = upscaling_task['id']
upscaling_task

{'id': 13,
 'title': 'Wind Turbine Detection',
 'label': 'openeo',
 'status': 'created'}

## Retrieve status of the upscaling task
We can now write a continuous monitoring process that fetches the status of the upscaling task and showcase the results on the map.

In [39]:
def add_cog_layer(cog_url, name=None, m=m):
    with rasterio.open(cog_url) as src:
        band = src.read(1).astype(np.float32)

        bounds = transform_bounds(src.crs, "EPSG:4326", *src.bounds)

        # Normalize 0–255
        band = 255 * (band - band.min()) / (band.max() - band.min())
        band = band.astype(np.uint8)

    # Convert to PNG data URI
    buf = io.BytesIO()
    Image.fromarray(band).save(buf, format="PNG")
    data_url = "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode("utf-8")

    bbox = ((bounds[1], bounds[0]), (bounds[3], bounds[2]))
    overlay = ImageOverlay(url=data_url, bounds=bbox, name=name or "Gray COG")
    m.add_layer(overlay)
    return overlay

In [40]:
m = Map(center=[center.y, center.x], zoom=zoom)
geo_json = GeoJSON(
    data={
        "type": "FeatureCollection",
        "features": []
    }
)
m.add_layer(geo_json)
display(m)

# Function to style jobs
def job_style(feature):
    color = {
        "created": "blue",
        "queued": "orange",
        "running": "yellow",
        "finished": "green",
        "canceled": "gray",
        "failed": "red"
    }.get(feature["properties"]["status"], "black")
    return {
        "color": color,
        "fillColor": color,
        "fillOpacity": 0.5
    }

# Keep track of processed jobs
processed_jobs = set()

async def show_results(job_id):
    async with httpx.AsyncClient() as client:
        result = await client.get(f"http://{dispatch_api}/unit_jobs/{job_id}/results")
        response = result.json()
        cog = response["assets"]["openEO.tif"]["href"]
        add_cog_layer(cog, name=f"Job {job_id}", m=m)
        return response

async def listen_for_updates():
    ws_url = f"ws://{dispatch_api}/ws/upscale_tasks/{upscaling_task_id}?interval=15"
    async with websockets.connect(ws_url) as websocket:
        while True:
            message = await websocket.recv()
            message = json.loads(message)
            if message.get("data"):
                features = []
                for job in message["data"]["jobs"]:
                    job_id = job["id"]
                    job_status = job["status"]
                    
                    # If the job is finished and not yet processed, fetch results
                    if job_status == "finished" and job_id not in processed_jobs:
                        processed_jobs.add(job_id)
                        await show_results(job_id)
                    elif job_status != "finished":
                        features.append({
                            "type": "Feature",
                            "geometry": job["parameters"]["spatial_extent"],
                            "properties": {
                                "status": job_status,
                            }
                        })
                    
                geo_json.data = {
                    "type": "FeatureCollection",
                    "features": features
                }
                geo_json.style_callback = job_style
                if message["data"]["status"] in ["finished", "canceled", "failed"]:
                    print(f"Job finished with status {message['data']['status']}")
                    websocket.close()
                    break

# Run the websocket listener in the notebook
await listen_for_updates()

Map(center=[47.27073518869459, 14.388191800967462], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Job finished with status failed


/var/folders/50/09_2zmx12zj6ks4fdl4y9wgc0000gn/T/ipykernel_92745/582752889.py:70: RuntimeWarning: coroutine 'Connection.close' was never awaited
  websocket.close()
